In [3]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt 
import numpy as np
import seaborn as sns
from datetime import datetime,timedelta
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from xgboost import plot_importance, plot_tree
from xgboost import XGBClassifier
import datetime
import holidays
from sklearn.preprocessing import LabelEncoder
import prophet
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor, plot_importance
from pre_processing_sy import *
import random
import os
import torch
from torch.autograd import Variable
import xgboost
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from random import randint
from catboost import CatBoostRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import VotingRegressor
# from sktime.forecasting.model_selection import temporal_train_test_split
# from sktime.utils.plotting import plot_series

In [4]:
def seed_everything(seed: int = 2024):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore
seed_everything(2024)

# TG 제외한 나머지 농산물

In [9]:
# 데이터 불러오기
train = pd.read_csv("dataset/train.csv")
test = pd.read_csv("dataset/test.csv")
trade = pd.read_csv("dataset/international_trade.csv")
submission = pd.read_csv("dataset/sample_submission.csv")


train_pre, test_pre = pre_all(train, test)

# 극 이상치 제거
tg_idx = train_pre[(train_pre["item"]=="TG") & (train_pre["price"]>20000)].index
rd_idx = train_pre[(train_pre["item"]=="RD") & (train_pre["price"]>5000)].index
bc_idx = train_pre[(train_pre["item"]=="BC") & (train_pre["price"]>8000)].index
cb_idx = train_pre[(train_pre["item"]=="CB") & (train_pre["price"]>2300)].index

train_pre.loc[tg_idx,"price"] = train_pre[(train_pre["item"]=="TG") & (train_pre["price"]!=0)]["price"].mean()
train_pre.loc[rd_idx,"price"] = train_pre[(train_pre["item"]=="RD") & (train_pre["price"]!=0)]["price"].mean()
train_pre.loc[bc_idx,"price"] = train_pre[(train_pre["item"]=="BC") & (train_pre["price"]!=0)]["price"].mean()
train_pre.loc[cb_idx,"price"] = train_pre[(train_pre["item"]=="CB") & (train_pre["price"]!=0)]["price"].mean()



# 감귤이 아닌것
print(f"train의 컬럼 : {train_pre.columns}")
print(f"test의 컬럼 : {test_pre.columns}")

train_tg = train_pre[train_pre["item"] !="TG"]
test_tg = test_pre[test_pre["item"] != "TG"]


Xy = pd.get_dummies(train_tg.sort_values(by = ["timestamp"]).reset_index(drop=True).drop(columns = ["supply"]), columns = ["item","corporation","location"])
answer = pd.get_dummies(test_tg.drop(columns = ["timestamp","supply","price"]), columns = [ "item","corporation","location"])
print(Xy.columns)


전처리 전 train 크기 : (59397, 7)
전처리 전 test 크기 : (1092, 5)
=================전처리 중=================
전처리 후 train 크기 : (59397, 15)
전처리 후 test 크기 : (1092, 15)
train의 컬럼 : Index(['ID', 'timestamp', 'item', 'corporation', 'location', 'supply', 'price',
       'year', 'month', 'day', 'week_day', 'year_month', 'week', 'week_num',
       'holiday'],
      dtype='object')
test의 컬럼 : Index(['ID', 'timestamp', 'item', 'corporation', 'location', 'supply', 'price',
       'year', 'month', 'day', 'week_day', 'year_month', 'week', 'week_num',
       'holiday'],
      dtype='object')
Index(['ID', 'timestamp', 'price', 'year', 'month', 'day', 'week_day',
       'year_month', 'week', 'week_num', 'holiday', 'item_BC', 'item_CB',
       'item_CR', 'item_RD', 'corporation_A', 'corporation_B', 'corporation_C',
       'corporation_D', 'corporation_E', 'corporation_F', 'location_J',
       'location_S'],
      dtype='object')


In [10]:
# # 모델 정의
cat = CatBoostRegressor(random_state = 2024, 
                            n_estimators = 1000, 
                            learning_rate = 0.01, 
                            depth = 10,
                            l2_leaf_reg = 3,
                            metric_period = 1000)

xgb = XGBRegressor(n_estimators = 1000, random_state = 2024, learning_rate = 0.01, max_depth = 10)


# voting
vote_model = VotingRegressor(
    estimators =[("cat",cat), ("xgb", xgb)]
)

vote_model.fit(Xy.drop(columns = ["timestamp", "ID","price"]), Xy["price"])

pred = vote_model.predict(answer.drop(columns = ["ID"]))
for idx in range(len(pred)):
    if pred[idx]<0:
        pred[idx]= 0
answer["answer"] = pred

answer[["ID","answer"]]

0:	learn: 934.3461033	total: 15.6ms	remaining: 15.5s
999:	learn: 376.6995081	total: 11.8s	remaining: 0us


,ID,answer
2,BC_B_J_20230304,2376.672901
4,BC_B_S_20230304,97.678967
6,BC_C_J_20230304,2385.800075
7,BC_A_S_20230304,2763.041513
10,BC_D_J_20230304,2771.454688
...,...,...
1085,RD_D_S_20230331,476.193654
1086,CR_C_J_20230331,1696.957199
1088,RD_D_J_20230331,387.570045
1089,CR_D_J_20230331,1818.944640


# TG 예측

In [11]:
# 데이터 불러오기
train = pd.read_csv("dataset/train.csv")
test = pd.read_csv("dataset/test.csv")
trade = pd.read_csv("dataset/international_trade.csv")
submission = pd.read_csv("dataset/sample_submission.csv")


train_pre, test_pre = pre_all(train, test)

# # 공휴일이지만 안쉬는 날 제외하기
no_holi = list(train_pre[(train_pre["item"] =="TG") &(train_pre["holiday"]==1) & (train_pre["price"]!=0)].groupby("timestamp").count().reset_index()["timestamp"])
noholi_idx = train_pre[train_pre["timestamp"].isin(no_holi)]["holiday"].index
for idx in noholi_idx:
    train_pre.loc[idx, "holiday"] = 0
    test_pre.loc[idx, "holiday"] = 0

print(f"train의 컬럼 : {train_pre.columns}")
print(f"test의 컬럼 : {test_pre.columns}")

train_tg = train_pre[train_pre["item"] =="TG"]
test_tg = test_pre[test_pre["item"] == "TG"]

Xy = pd.get_dummies(train_tg.sort_values(by = ["timestamp", "corporation","location"]).reset_index(drop=True).drop(columns = ["item","supply"]), columns = [ "corporation","location"])
answer_tg = pd.get_dummies(test_tg.drop(columns = ["timestamp","supply","price","item",]), columns = [ "corporation","location"])
print(Xy.columns)

# 종속변수 루트값
Xy["price"] = np.sqrt(Xy["price"])


전처리 전 train 크기 : (59397, 7)
전처리 전 test 크기 : (1092, 5)
=================전처리 중=================
전처리 후 train 크기 : (59397, 15)
전처리 후 test 크기 : (1092, 15)
train의 컬럼 : Index(['ID', 'timestamp', 'item', 'corporation', 'location', 'supply', 'price',
       'year', 'month', 'day', 'week_day', 'year_month', 'week', 'week_num',
       'holiday'],
      dtype='object')
test의 컬럼 : Index(['ID', 'timestamp', 'item', 'corporation', 'location', 'supply', 'price',
       'year', 'month', 'day', 'week_day', 'year_month', 'week', 'week_num',
       'holiday'],
      dtype='object')
Index(['ID', 'timestamp', 'price', 'year', 'month', 'day', 'week_day',
       'year_month', 'week', 'week_num', 'holiday', 'corporation_A',
       'corporation_B', 'corporation_C', 'corporation_D', 'corporation_E',
       'location_J', 'location_S'],
      dtype='object')


In [12]:
# 예측 및 
n_estimators =1000
lrs = 0.05
max_depths = 10
l2_leaf_reg = 3

cat = CatBoostRegressor(random_state = 2024, 
                                n_estimators = n_estimators, 
                                learning_rate = lrs, 
                                depth = max_depths, 
                                l2_leaf_reg = l2_leaf_reg,
                                metric_period = 1000)

cat.fit(Xy.drop(columns = ["timestamp", "ID","price"]), Xy["price"])

pred = cat.predict(answer_tg.drop(columns = ["ID"]))
for idx in range(len(pred)):
    if pred[idx]<0:
        pred[idx]= 0
answer_tg["answer"] = np.power(pred,2)

answer_tg[["ID","answer"]]

0:	learn: 30.2823698	total: 12.6ms	remaining: 12.6s
999:	learn: 11.0655757	total: 9.38s	remaining: 0us


,ID,answer
0,TG_A_J_20230304,2742.172780
1,TG_E_S_20230304,3354.003496
3,TG_E_J_20230304,760.581106
5,TG_D_S_20230304,3769.615969
8,TG_D_J_20230304,53.451592
...,...,...
1074,TG_D_S_20230331,4727.895634
1077,TG_A_S_20230331,5403.967583
1079,TG_E_S_20230331,4715.201735
1087,TG_A_J_20230331,6813.935154


In [17]:
total = pd.concat([answer_tg[["ID","answer"]],answer[["ID","answer"]]])

In [19]:
total.sort_values("ID")

,ID,answer
17,BC_A_J_20230304,2840.355945
64,BC_A_J_20230305,98.708493
82,BC_A_J_20230306,2597.832506
128,BC_A_J_20230307,2838.838130
189,BC_A_J_20230308,2847.775354
...,...,...
928,TG_E_S_20230327,4533.301873
957,TG_E_S_20230328,4406.550254
1006,TG_E_S_20230329,4319.457891
1044,TG_E_S_20230330,4636.905040
